In [ ]:
!pip install tensorflow==2.1.0

ERROR: Could not find a version that satisfies the requirement tensorflow==2.1.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0)
ERROR: No matching distribution found for tensorflow==2.1.0


Importing libraries

In [ ]:
!pip install tflearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=46efbeb0abeca173b7f5af5625d7f86440309706920b508c64bc04b7699155d0
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [ ]:
import os
import tflearn
import tensorflow.compat.v1 as tf
import numpy
import random

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import json
with open('/content/drive/MyDrive/Colab Notebooks/Bankbot/intents.json') as file:
    data = json.load(file)

ImportError: ignored

In [ ]:
nltk.download('punkt')

In [ ]:
testing_data = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        testing_data.append((pattern, intent['tag']))

Extracting data

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        docs_x.append(w)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

Stemming words

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

Developing model

In [ ]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Training model

In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Streaming output truncated to the last 5000 lines.
Training Step: 1290  | total loss: 0.23469 | time: 0.026s
| Adam | epoch: 258 | loss: 0.23469 - acc: 1.0000 -- iter: 36/36
--
Training Step: 1291  | total loss: 0.23469 | time: 0.004s
| Adam | epoch: 259 | loss: 0.23469 - acc: 1.0000 -- iter: 08/36
Training Step: 1292  | total loss: 0.23237 | time: 0.008s
| Adam | epoch: 259 | loss: 0.23237 - acc: 1.0000 -- iter: 16/36
Training Step: 1293  | total loss: 0.23543 | time: 0.011s
| Adam | epoch: 259 | loss: 0.23543 - acc: 1.0000 -- iter: 24/36
Training Step: 1294  | total loss: 0.23518 | time: 0.016s
| Adam | epoch: 259 | loss: 0.23518 - acc: 1.0000 -- iter: 32/36
Training Step: 1295  | total loss: 0.23117 | time: 0.023s
| Adam | epoch: 259 | loss: 0.23117 - acc: 1.0000 -- iter: 36/36
--
Training Step: 1296  | total loss: 0.22745 | time: 0.005s
| Adam | epoch: 260 | loss: 0.22745 - acc: 1.0000 -- iter: 08/36
Training Step: 1297  | total loss: 0.23270 | time: 0.010s
| Adam | epoch: 260 | lo

Prediction

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

def chat():
    print("Start talking with the bot")
    while True:
        inp = input("User: ")
        if inp.lower() in ["bye", "quit", "exit"]:
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[0][results_index] > 0.7:  # Adjust the confidence threshold as needed
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print("Bot: " + random.choice(responses))
        else:
            print("Bot: Sorry, I could not understand.")

chat()

Start talking with the bot
Bot: Sorry, I could not understand.
Bot: Here is the list of required documents to apply for home loan: 1. 3 photographs passport sized 2. Identify proof 3. Residence proof 4. Bank Account Statement/Passbook for last 6 months 5. Signature verification by bankers of the applicant 6. Liabilities statement and Personal Assets 7. Property detailed documents.
User: bro movie
Bot: Sorry, I could not understand.
